## Points to Paths in Python

Getting ready for MovingPandas
I’ll start by loading the necessary libraries and creating a helper function for mapping as well as some defaults for annotating points on the maps. Then i will load some data converted from raw GPS data as described in the previous article.

python3 -m venv maps
source maps/bin/activate

In [4]:
# %pip install pandas geopandas movingpandas matplotlib

%pip install contextlily

ERROR: Could not find a version that satisfies the requirement contextlily (from versions: none)
ERROR: No matching distribution found for contextlily
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd 
import geopandas as gpd 
import movingpandas as mpd 
import matplotlib.pyplot as plt
import contextily as ctx
from datetime import datetime, timedelta
from geopandas.tools import reverse_geocode, geocode
import warnings
warnings.filterwarnings('ignore')
 
def t_plot(traj, 
            figsize=(6,8), 
            source=ctx.providers.CartoDB.Voyager, 
            title=None):
    f, ax = plt.subplots(figsize=figsize)
    traj.plot(ax=ax, lw=4, color='chartreuse')
    ctx.add_basemap(ax, crs=traj.crs, 
                    source=source)
    ax.set_axis_off()
    if title: 
        ax.set_title(title)
    else:
        ax.set_title(
            f'Walk in {traj.df.Name[0]} on {traj.df.index[0].strftime('%x')}'
        )
    return f, ax
 
annot_props = dict(
    xytext=(3, 1), 
    textcoords="offset fontsize", 
    c='r', weight='bold', ha='center',
    arrowprops=dict(arrowstyle='-', ec='orange')
)
 
df = pd.read_csv('data/b3/combined.csv')
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(x=df.Lon, y=df.Lat), 
    crs=4269
).to_crs(32111)

/Users/ulrike_imac_air/projects/maps/Openstreetmap/maps/lib/python3.13/site-packages/movingpandas/__init__.py:41: UserWarning: Missing optional dependencies. To use the trajectory smoother classes please install Stone Soup (see https://stonesoup.readthedocs.io/en/latest/#installation).
  warnings.warn(e.msg, UserWarning)


ModuleNotFoundError: No module named 'contextily'

MovingPandas requires an integer id for each walk, which I don’t have, so I’ll create one based on the string id. I’ll also convert the Time from a generic object to datetime64.

In [ ]:
idList = list(gdf.groupby(['Id']).nunique().reset_index().Id)
for i, track in enumerate(idList):
    gdf.loc[gdf.Id == track, 'trajectory_id'] = i
gdf['Time'] = pd.to_datetime(gdf.Time)
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   Id             3044 non-null   object             
 1   Name           3044 non-null   object             
 2   Lat            3044 non-null   float64            
 3   Lon            3044 non-null   float64            
 4   Elev           3044 non-null   int64              
 5   Time           3044 non-null   datetime64[ns, UTC]
 6   Temp           3044 non-null   float64            
 7   Weather        3044 non-null   int64              
 8   geometry       3044 non-null   geometry           
 9   trajectory_id  3044 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(4), geometry(1), int64(2), object(2)
memory usage: 237.9+ KBt

One of the walks in the collection was in New York City, and the rest in New Jersey. Since the locations are pretty far apart, I’ll split out the NYC walk from the others.

In [ ]:
gdf_nj = gdf.loc[gdf.Name != 'New York']
gdf_ny = gdf.loc[gdf.Name == 'New York']
gdf_nj.shape, gdf_ny.shape